In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import keras.backend as K
import tensorflow as tf

def categorical_focal_loss(gamma=2.0, alpha=0.25):
    """
    Implementation of Focal Loss from the paper in multiclass classification
    Formula:
        loss = -alpha*((1-p)^gamma)*log(p)
    Parameters:
        alpha -- the same as wighting factor in balanced cross entropy
        gamma -- focusing parameter for modulating factor (1-p)
    Default value:
        gamma -- 2.0 as mentioned in the paper
        alpha -- 0.25 as mentioned in the paper
    """
    def focal_loss(y_true, y_pred):
        # Define epsilon so that the backpropagation will not result in NaN
        # for 0 divisor case
        epsilon = K.epsilon()
        # Add the epsilon to prediction value
        #y_pred = y_pred + epsilon
        # Clip the prediction value
        y_pred = K.clip(y_pred, epsilon, 1.0-epsilon)
        # Calculate cross entropy
        cross_entropy = -y_true*K.log(y_pred)
        # Calculate weight that consists of  modulating factor and weighting factor
        weight = alpha * y_true * K.pow((1-y_pred), gamma)
        # Calculate focal loss
        loss = weight * cross_entropy
        # Sum the losses in mini_batch
        loss = K.sum(loss, axis=1)
        return loss
    
    return focal_loss


In [69]:
import numpy as np
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation
from keras.layers import Embedding
from keras.layers import Conv1D, GlobalAveragePooling1D, MaxPooling1D
from keras.optimizers import SGD
from sklearn.model_selection import train_test_split
from keras.callbacks import ModelCheckpoint 
from datetime import datetime
import tensorflow as tf
from keras.utils.np_utils import to_categorical
from keras.callbacks import ModelCheckpoint 
from keras.callbacks import EarlyStopping
from keras.regularizers import l2
from keras.layers import Input
from keras.layers import Flatten
from keras.layers import Dropout
from keras.models import Model

from keras.layers.merge import concatenate
from imblearn.over_sampling import SMOTE

from sklearn.preprocessing import LabelEncoder
import os
X = np.load('/content/drive/MyDrive/test_co/feat (1).npy')
y=np.load('/content/drive/MyDrive/test_co/label.npy')


s = SMOTE(k_neighbors=3)
X,y=s.fit_resample(X,y)



# Split data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2,stratify=y, random_state = 233)

# Convert label to onehot encoding
le = LabelEncoder()

y_train = to_categorical(le.fit_transform(y_train))
y_test = to_categorical(le.fit_transform(y_test))
# y_over = to_categorical(le.fit_transform(y_over))
a=len(np.where(y_train==0)[0]) 
b=len(np.where(y_train==1)[0])
c=len(np.where(y_train==2)[0]), 
# Make 2-dim into 3-dim array for input for model training
X_train = np.expand_dims(X_train, axis=2)
X_test = np.expand_dims(X_test, axis=2)

n_timesteps=X_train.shape[1]
n_features = X_train.shape[2]


inputs1 = Input(shape=(n_timesteps,n_features))
conv1 = Conv1D(filters=1024, kernel_size=7, activation='relu')(inputs1)
conv1_2 = Conv1D(filters=512, kernel_size=7, activation='relu')(conv1)
pool1 = MaxPooling1D(pool_size=2)(conv1_2)
conv1_3 = Conv1D(filters=256, kernel_size=7, activation='relu')(pool1)
conv1_4= Conv1D(filters=128, kernel_size=7, activation='relu')(conv1_3)
glob_1 = GlobalAveragePooling1D()(conv1_4)
drop1 = Dropout(0.4)(glob_1)
flat1 = Flatten()(drop1)
# head 2

inputs2 = Input(shape=(n_timesteps,n_features))
conv2 = Conv1D(filters=1024, kernel_size=7, activation='relu')(inputs2)
conv2_2 = Conv1D(filters=512, kernel_size=7, activation='relu')(conv2)
pool2 = MaxPooling1D(pool_size=2)(conv2_2)
conv2_3 = Conv1D(filters=256, kernel_size=7, activation='relu')(pool1)
glob_2 = GlobalAveragePooling1D()(conv2_3)
drop2 = Dropout(0.4)(glob_2)
flat2 = Flatten()(drop2)
# head 3
inputs3 = Input(shape=(n_timesteps,n_features))
conv3 = Conv1D(filters=64, kernel_size=21, activation='relu')(inputs3)
drop3 = Dropout(0.5)(conv3)
pool3 = MaxPooling1D(pool_size=2)(drop3)
flat3 = Flatten()(pool3)
# merge
merged = concatenate([flat1, flat2, flat3])
# interpretation
dense1 = Dense(256, activation = 'relu')(merged)
dense2 = Dense(128, activation = 'relu')(dense1)
dense3 = Dense(64, activation='relu')(dense2)
outputs = Dense(3, activation='softmax')(dense3)
model = Model(inputs=[inputs1, inputs2, inputs3], outputs=outputs)
# save a plot of the model
print(model.summary())
batch_size=64
epochs=100 # Epochs are tunable

#
lr_schedule = keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate=0.00035,
    decay_steps=1000,
    decay_rate=0.9)


opt = keras.optimizers.Adam(learning_rate=lr_schedule)

model.compile(loss=[categorical_focal_loss()],
              optimizer=opt,
              metrics=['accuracy'])

model_file = 'file-train-nb3.hdf5'
models_path = '/content/drive/MyDrive/dd/'
model_path = os.path.join(models_path, model_file)


# Save checkpoints
checkpointer = ModelCheckpoint(filepath=model_path, 
                               verbose=1, 
                               save_best_only=True)
es = EarlyStopping(patience=10)
start = datetime.now()
history = model.fit([X_train,X_train,X_train], 
                    y_train, 
                    batch_size=batch_size, 
                    epochs=epochs, 
                    validation_split=1/12.,
                    callbacks=[checkpointer,es], 
                    verbose=1,
                    )

duration = datetime.now() - start
print("Training completed in time: ", duration)


# Compute Accuracy and Loss


/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarning)


Model: "model_18"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_55 (InputLayer)           [(None, 193, 1)]     0                                            
__________________________________________________________________________________________________
conv1d_144 (Conv1D)             (None, 187, 1024)    8192        input_55[0][0]                   
__________________________________________________________________________________________________
conv1d_145 (Conv1D)             (None, 181, 512)     3670528     conv1d_144[0][0]                 
__________________________________________________________________________________________________
max_pooling1d_54 (MaxPooling1D) (None, 90, 512)      0           conv1d_145[0][0]                 
___________________________________________________________________________________________

In [70]:
score, acc = model.evaluate([X_test,X_test,X_test], y_test, batch_size=8)

614/614 [==============================] - 4s 7ms/step - loss: 0.0290 - accuracy: 0.9126


In [71]:
from sklearn.metrics import confusion_matrix,f1_score
y_probs = model.predict([X_test,X_test,X_test], verbose=0)
yhat_probs = np.argmax(y_probs, axis=1)

confusion_matrix(y_test.argmax(axis=1), y_probs.argmax(axis=1))

array([[1413,   97,  126],
       [  19, 1615,    2],
       [ 179,    6, 1451]])

In [72]:
f1_score(y_test.argmax(axis=1), y_probs.argmax(axis=1),average='micro')

0.9125916870415648

lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate,
    decay_steps=100000,
    decay_rate=0.96,
    staircase=True)

In [37]:
model.save_weights("/content/drive/MyDrive/test_co/aks.h5")